In [6]:
import os
import pandas as pd
import requests

Deskripsi label:
1. 0 (PIP) → Kinerja buruk, butuh perbaikan segera.
2. 1 (Need Improvement) → Performa kurang baik, tapi belum sampai tahap PIP.
3. 2 (Fully Meets) → Memenuhi harapan pekerjaan dengan baik.
4. 3 (Exceeded) → Melebihi ekspektasi dalam kinerja.


In [7]:
df = pd.read_csv("data/Cleaned_HR_Data_Analysis.csv")

In [8]:
from pprint import PrettyPrinter
pp = PrettyPrinter()
pp.pprint(requests.get("https://wahyuazizi-mlops.up.railway.app/v1/models/employee-performance-model"))

<Response [200]>


In [9]:

CATEGORICAL_FEATURES = {
    "BusinessUnit": 10,
    "EmployeeStatus": 2,
    "EmployeeType": 3,
    "PayZone": 3,
    "EmployeeClassificationType": 3,
    "DepartmentType": 6,
    "GenderCode": 2,
    "RaceDesc": 5,
    "MaritalDesc": 4,
    "TrainingType": 2,
    "TrainingOutcome": 4
}

NUMERICAL_FEATURES = [
    "CurrentEmployeeRating",
    "EngagementScore",
    "SatisfactionScore",
    "Work-LifeBalanceScore",
    "TrainingDurationDays",
    "TrainingCost",
    "Age"
]

LABEL_KEY = "PerformanceScore"


In [10]:
# Hanya pilih kolom yang diperlukan
selected_columns = list(CATEGORICAL_FEATURES.keys()) + NUMERICAL_FEATURES + [LABEL_KEY]
df = df[selected_columns]

save_dir = "data"

# Simpan file CSV dalam folder 'data'
save_path = os.path.join(save_dir, "Cleaned_HR_Data_Analysis.csv")
df.to_csv(save_path, index=False)

print(f"File disimpan di {save_path}")

File disimpan di data\Cleaned_HR_Data_Analysis.csv


In [11]:
import tensorflow as tf
import json
import base64
import requests

# Mapping hasil prediksi angka ke string
LABEL_MAPPING = {
    0: "PIP - Kinerja buruk, butuh perbaikan segera.",
    1: "Need Improvement - Performa kurang baik, tapi belum sampai tahap PIP.",
    2: "Fully Meets - Memenuhi harapan pekerjaan dengan baik.",
    3: "Exceeded - Melebihi ekspektasi dalam kinerja."
}

# Data contoh yang akan dikirim ke model
input_data = {
    "BusinessUnit": "NEL",
    "EmployeeStatus": "Active",
    "EmployeeType": "Contract",
    "PayZone": "Zone B",
    "EmployeeClassificationType": "Full-Time",
    "DepartmentType": "Sales",
    "GenderCode": "Male",
    "RaceDesc": "White",
    "MaritalDesc": "Widowed",
    "TrainingType": "Internal",
    "TrainingOutcome": "Incomplete",
    "CurrentEmployeeRating": 5,  # Integer
    "EngagementScore": 3,  # Integer
    "SatisfactionScore": 4,  # Integer
    "Work-LifeBalanceScore": 5,  # Integer
    "TrainingDurationDays": 3,  # Integer
    "TrainingCost": 442.56,  # Float
    "Age": 48  # Integer
}

# Fitur yang harus dikirim sebagai int64
INT_FEATURES = [
    "CurrentEmployeeRating",
    "EngagementScore",
    "SatisfactionScore",
    "Work-LifeBalanceScore",
    "TrainingDurationDays",
    "Age"
]

# Fitur yang dikirim sebagai float
FLOAT_FEATURES = ["TrainingCost"]

def prepare_json(inputs: dict):
    feature_spec = {}

    for key, value in inputs.items():
        if key in INT_FEATURES:
            feature_spec[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[int(value)]))
        elif key in FLOAT_FEATURES:
            feature_spec[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[float(value)]))
        elif isinstance(value, str):  # Fitur kategorikal (string) dikonversi ke bytes
            feature_spec[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

    # Serialize data menjadi TFRecord (base64 encoding)
    example = tf.train.Example(features=tf.train.Features(feature=feature_spec))
    serialized_example = example.SerializeToString()

    # Format sesuai dengan request model di Railway
    result = [{"examples": {"b64": base64.b64encode(serialized_example).decode()}}]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

# Konversi data ke format JSON
json_data = prepare_json(input_data)

# Endpoint model di Railway
endpoint = "https://wahyuazizi-mlops.up.railway.app/v1/models/employee-performance-model:predict"

# Kirim request ke model yang telah dideploy
response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})

# Ambil hasil prediksi
if response.status_code == 200:
    prediction = response.json().get("predictions")
    if prediction:
        predicted_class = prediction[0].index(max(prediction[0]))  # Mengambil indeks dengan nilai tertinggi
        predicted_label = LABEL_MAPPING.get(predicted_class, "Unknown")  # Konversi angka ke deskripsi label
        print(f"Predicted Performance Score: {predicted_class} ({predicted_label})")
    else:
        print("Error: No predictions found in response.")
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")


Predicted Performance Score: 1 (Need Improvement - Performa kurang baik, tapi belum sampai tahap PIP.)
